# VLM-Based Fraud Detection Testing

This notebook demonstrates the VLM-based fraud detection system with interactive examples.

## Setup

First, import the necessary modules and configure API keys.

In [ ]:
import sys
from pathlib import Path
import pprint
import asyncio

# Get the notebook's directory and navigate up to the project root
notebook_dir = Path().resolve()
project_root = notebook_dir.parent  # Goes up two levels 

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.vlm_models import VLMAnalyzer, VLMConfig

### Configure API Keys

Set your API keys (uncomment and fill in):

## 1. Basic Configuration

In [ ]:
# Create configuration
config = VLMConfig(model_provider="claude-4-5-sonnet", max_tokens=4000)

print(f"Model: {config.model_provider}")
print(f"OpenRouter Model ID: {config.get_openrouter_model_id()}")
print(f"Temperature: {config.temperature}")
print(f"Max Tokens: {config.max_tokens}")

## 2. Initialize Analyzer

In [ ]:
analyzer = VLMAnalyzer(config=config)
print("VLM Analyzer initialized successfully")

## 3. Analyze a Fraudulent Check

In [ ]:
# Analyze a fraudulent check with Photoshop manipulation
fraud_result = analyzer.analyze_image("/Users/Pablo.Vargas2/Documents/isitreal-pablo/data/fraudulent/tipex_1.jpg")

In [ ]:
# Access structured results
analysis = fraud_result.analysis

print(f"Is Fraudulent: {analysis.is_fraudulent}")
print(f"Confidence: {analysis.overall_confidence}")
print(f"Fraud Score: {analysis.fraud_likelihood_score}/100")
print(f"Recommendation: {analysis.recommendation}")
print(f"\nProcessing Time: {fraud_result.processing_time:.2f}s")

In [ ]:
# View suspicious regions
print(f"Suspicious Regions Found: {len(analysis.suspicious_regions)}\n")

for i, region in enumerate(analysis.suspicious_regions, 1):
    print(f"Region {i}:")
    print(f"  Location: {region.location}")
    print(f"  Type: {region.fraud_type}")
    print(f"  Confidence: {region.confidence}")
    print(f"  Description: {region.description}")
    print()

In [ ]:
# View detailed analysis
print("Detailed Analysis:")
print("─" * 80)
print(analysis.detailed_analysis)
print("─" * 80)

In [ ]:
# View key indicators
print("Key Indicators:")
for i, indicator in enumerate(analysis.key_indicators, 1):
    print(f"  {i}. {indicator}")

## 4. Analyze a Legitimate Check

In [ ]:
# Analyze a legitimate check
clean_result = analyzer.analyze_image("/Users/Pablo.Vargas2/Documents/isitreal-pablo/data/non_fraud/blurry.png")

In [ ]:
# Compare results
print("Comparison: Fraudulent vs Clean Check\n")
print(f"{'Metric':<25} | {'Fraudulent Check':<20} | {'Clean Check':<20}")
print("─" * 70)
print(f"{'Is Fraudulent':<25} | {str(fraud_result.analysis.is_fraudulent):<20} | {str(clean_result.analysis.is_fraudulent):<20}")
print(f"{'Fraud Score':<25} | {fraud_result.analysis.fraud_likelihood_score:<20.1f} | {clean_result.analysis.fraud_likelihood_score:<20.1f}")
print(f"{'Confidence':<25} | {fraud_result.analysis.overall_confidence:<20} | {clean_result.analysis.overall_confidence:<20}")
print(f"{'Recommendation':<25} | {fraud_result.analysis.recommendation:<20} | {clean_result.analysis.recommendation:<20}")

## 5. Batch Analysis

In [ ]:
# Analyze multiple checks
image_paths = [
    "../data/fraudulent/photoshop.jpg",
    "../data/fraudulent/tipex_1.jpg",
    "../data/non_fraud/clear_stamp.png",
    "../data/non_fraud/blurry.png"
]

batch_results = analyzer.analyze_batch(image_paths)

In [ ]:
# Summary table
import pandas as pd

summary_data = []
for result in batch_results:
    summary_data.append({
        "Image": Path(result.image_path).name,
        "Fraudulent": "🚨 YES" if result.analysis.is_fraudulent else "✅ NO",
        "Score": f"{result.analysis.fraud_likelihood_score:.1f}/100",
        "Confidence": result.analysis.overall_confidence,
        "Recommendation": result.analysis.recommendation.upper(),
        "Time (s)": f"{result.processing_time:.2f}"
    })

df = pd.DataFrame(summary_data)
display(df)

## 6. Model Comparison

In [ ]:
# Compare different models on the same image
test_image = "../data/fraudulent/photoshop.jpg"

models_to_test = [
    "gpt-4o",
    "claude-4-5-haiku",
    "gemini-2.5-flash"
]

model_results = {}

for model in models_to_test:
    print(f"\nTesting: {model}")
    try:
        config = VLMConfig(model_provider=model, save_results=False)
        analyzer = VLMAnalyzer(config=config)
        result = analyzer.analyze_image(test_image)
        model_results[model] = result
    except Exception as e:
        print(f"  ❌ Failed: {e}")

In [ ]:
# Compare model results
comparison_data = []
for model, result in model_results.items():
    comparison_data.append({
        "Model": model,
        "Fraudulent": "YES" if result.analysis.is_fraudulent else "NO",
        "Score": f"{result.analysis.fraud_likelihood_score:.1f}",
        "Confidence": result.analysis.overall_confidence,
        "Time (s)": f"{result.processing_time:.2f}",
        "# Regions": len(result.analysis.suspicious_regions)
    })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

## 8. Integration with Traditional CV

In [ ]:
# Compare VLM with traditional CV methods
from src.traditional_cv import AnomalyAnalyzer, AnalysisConfig

# Traditional CV analysis
cv_config = AnalysisConfig(save_visualizations=False)
cv_analyzer = AnomalyAnalyzer(config=cv_config)
cv_result = cv_analyzer.analyze_image(test_image)

print("Traditional CV vs VLM Comparison\n")
print(f"{'Method':<20} | {'Score':<15} | {'Classification':<20}")
print("─" * 60)
print(f"{'Traditional CV':<20} | {cv_result.overall_score:<15.1f} | {cv_result.fraud_likelihood:<20}")
print(f"{'VLM (GPT-4o-mini)':<20} | {fraud_result.analysis.fraud_likelihood_score:<15.1f} | {'Fraudulent' if fraud_result.analysis.is_fraudulent else 'Clean':<20}")

# Combined score (weighted average)
combined_score = cv_result.overall_score * 0.4 + fraud_result.analysis.fraud_likelihood_score * 0.6
print(f"{'Combined (40/60)':<20} | {combined_score:<15.1f} | {'Fraudulent' if combined_score > 50 else 'Clean':<20}")

## Summary

This notebook demonstrated:
1. ✅ VLM configuration and initialization
2. ✅ Single image fraud analysis
3. ✅ Structured output validation
4. ✅ Batch processing
5. ✅ Model comparison
6. ✅ Integration with traditional CV methods

The VLM-based approach provides:
- 🎯 Semantic understanding of check structure
- 📍 Specific suspicious region identification
- 💡 Human-readable explanations
- 🔄 Easy integration with traditional methods